In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [3]:
resorts = pd.read_csv("dataset/resorts.csv")
resorts

,ski_resort_name,skiable_acres,Weekly Pass Price (AUD),Direct Public Coach,Beginner%,Intermediate%,Advanced%,Expert%
0,Falls Creek Ski Resort,1100.0,1239,1,17,60,23,0
1,Hotham Ski Resort,790.0,1400,1,8,38,43,11
2,Mt Buller Ski Resort,740.0,1260,1,5,41,51,4
3,Perisher Ski Resort,3076.0,1281,1,22,60,18,0
4,Thredbo Ski Resort,1186.0,1260,1,16,67,17,0
5,Mt Baw Baw Ski Resort,86.0,555,0,25,64,11,0
6,Selwyn Ski Resort,110.0,833,0,29,42,21,8
7,Charlotte Pass Ski Resort,124.0,1148,0,32,28,0,40
8,Mt Stirling Ski Resort,NaN,469,0,20,60,20,0


In [6]:
def iqr(x):
    q75, q25 = np.nanpercentile(x, [75, 25])
    return q75 - q25

def logistic_cost(x, theta, gamma):
    # higher price/visitors -> lower utility in (0,1)
    return 1.0 / (1.0 + np.exp((x - theta) / (gamma if gamma != 0 else 1e-8)))
def size_utility(size):
    # log-scaled min-max to [0,1]
    s = np.array(size, dtype=float)
    log_s = np.log(s)
    mn, mx = np.nanmin(log_s), np.nanmax(log_s)
    return (log_s - mn) / (mx - mn)

In [7]:
resorts['skiable_acres'] = size_utility(resorts['skiable_acres'])

In [10]:
def level_norm(col):
    arr = np.array(col, dtype=float)   # convert to NumPy array
    return (arr / 100.0) .round(2)
resorts['Beginner%'] = level_norm(resorts['Beginner%'])
resorts['Intermediate%'] = level_norm(resorts['Intermediate%'])
resorts['Advanced%'] = level_norm(resorts['Advanced%'])
resorts['Expert%'] = level_norm(resorts['Expert%'])
resorts

,ski_resort_name,skiable_acres,Weekly Pass Price (AUD),Direct Public Coach,Beginner%,Intermediate%,Advanced%,Expert%
0,Falls Creek Ski Resort,0.712522,1239,1,0.17,0.60,0.23,0.00
1,Hotham Ski Resort,0.619978,1400,1,0.08,0.38,0.43,0.11
2,Mt Buller Ski Resort,0.601700,1260,1,0.05,0.41,0.51,0.04
3,Perisher Ski Resort,1.000000,1281,1,0.22,0.60,0.18,0.00
4,Thredbo Ski Resort,0.733566,1260,1,0.16,0.67,0.17,0.00
5,Mt Baw Baw Ski Resort,0.000000,555,0,0.25,0.64,0.11,0.00
6,Selwyn Ski Resort,0.068809,833,0,0.29,0.42,0.21,0.08
7,Charlotte Pass Ski Resort,0.102301,1148,0,0.32,0.28,0.00,0.40
8,Mt Stirling Ski Resort,NaN,469,0,0.20,0.60,0.20,0.00


In [49]:
def visitors_utility(mu_vis, sd_vis, theta, gamma, lambda_V=0.3):
    mu = np.clip(mu_vis, 0, None)
    sd = np.where(np.isnan(sd_vis), 0.0, sd_vis)
    cv = np.where(mu > 0, sd / np.maximum(mu, 1e-8), 0.0)
    base = logistic_cost(mu, theta, gamma)
    return np.clip(base * (1.0 - lambda_V * cv), 0.0, 1.0)
    
def estimate_sd_from_interval(yhat_lower, yhat_upper):
    """Approximate 1σ from ~95% prediction interval."""
    return (np.asarray(yhat_upper) - np.asarray(yhat_lower)) / 4.0
def compute_visitor_utilities(csv_path, lambda_V=0.3):
    """Read a forecast CSV and compute weekly visitor utilities."""
    df = pd.read_csv(csv_path)
    mu_vis = df["yhat"].values
    sd_vis = estimate_sd_from_interval(df["yhat_lower"].values, df["yhat_upper"].values)

    # theta = median(mu), gamma = IQR/1.35 (robust σ) else fallback
    iqr_val = iqr(mu_vis)
    theta_V = float(np.nanmedian(mu_vis))
    gamma_V = float(iqr_val / 1.35) if iqr_val > 0 else float(max(np.nanstd(mu_vis), 1.0))

    util = visitors_utility(mu_vis, sd_vis, theta_V, gamma_V, lambda_V=lambda_V)

    # add to dataframe
    out = df.copy()
    stem = Path(csv_path).stem
    mountain = re.search(r'([^/]+)_forecast', stem).group(1)
    out['M_name'] = mountain
    out["visitor_utility"] = util
    return out

def refine(csv_path):
    temp = compute_visitor_utilities(csv_path)
    cols = ['yhat', 'yhat_lower','yhat_upper']
    temp = temp.drop(columns=cols)
    return temp

Fc = refine("dataset/Falls_Creek_forecast_2025.csv")
BB = refine("dataset/Mt._Baw_Baw_forecast_2025.csv")
Buller = refine("dataset/Mt._Buller_forecast_2025.csv")
Charlotte = refine("dataset/Charlotte_Pass_forecast_2025.csv")
Hotham = refine("dataset/Mt._Hotham_forecast_2025.csv")
Stirling = refine("dataset/Mt._Stirling_forecast_2025.csv")
Perisher = refine("dataset/Perisher_forecast_2025.csv")
Selwyn = refine("dataset/Selwyn_forecast_2025.csv")
Thredbo = refine("dataset/Thredbo_forecast_2025.csv")
Fc

,ds,M_name,visitor_utility
0,2025-06-02,Falls_Creek,0.740217
1,2025-06-09,Falls_Creek,0.731972
2,2025-06-16,Falls_Creek,0.742280
3,2025-06-23,Falls_Creek,0.677048
4,2025-06-30,Falls_Creek,0.490229
5,2025-07-07,Falls_Creek,0.383292
6,2025-07-14,Falls_Creek,0.409158
7,2025-07-21,Falls_Creek,0.465083
8,2025-07-28,Falls_Creek,0.446297
9,2025-08-04,Falls_Creek,0.363234


In [14]:
def minmax01(x):
    mn, mx = np.nanmin(x), np.nanmax(x)
    return (x - mn) / (mx - mn) if mx > mn else np.zeros_like(x)

def entropy_weights(Z):
    Z = np.asarray(Z, dtype=float)
    col_sums = Z.sum(axis=0)
    col_sums = np.where(col_sums == 0, 1e-12, col_sums)
    P = Z / col_sums
    P_safe = np.where(P <= 0, 1e-12, P)
    n = Z.shape[0]
    e = - (P_safe * np.log(P_safe)).sum(axis=0) / np.log(n)
    d = 1 - e
    return d / d.sum() if d.sum() > 0 else np.ones(Z.shape[1]) / Z.shape[1]

def topsis_scores(Z, w):
    V = Z * w
    v_plus, v_minus = V.max(axis=0), V.min(axis=0)
    D_plus = np.sqrt(((V - v_plus)**2).sum(axis=1))
    D_minus = np.sqrt(((V - v_minus)**2).sum(axis=1))
    return D_minus / (D_plus + D_minus + 1e-12)

In [25]:
snow = pd.read_csv("dataset/weekly_avg_mm_per_day_2026.csv")
snow = snow.melt(id_vars=["WeekStart"], 
                  var_name="M_name", 
                  value_name="avg_mm")
def snow_utility(avg_snow_mm_day, tau=None, p=2.0):
    avg_snow = np.clip(np.array(avg_snow_mm_day, dtype=float), 0, None)
    if tau is None:
        tau = np.nanmedian(avg_snow)  # <-- use median of the dataset
    return 1 - np.exp(- (avg_snow / tau) ** p)
snow["snow_utility"] = snow_utility(snow["avg_mm"], tau=None, p=2.0)

In [18]:
resorts["u_size"] = minmax01(np.log(resorts["skiable_acres"]+1e-8))
resorts["u_price"] = 1 / (1 + np.exp((resorts["Weekly Pass Price (AUD)"] 
                                     - resorts["Weekly Pass Price (AUD)"].median()) 
                                    / (resorts["Weekly Pass Price (AUD)"].std()+1e-8)))
resorts["u_access"] = resorts["Direct Public Coach"].astype(float)
terrain_cols = ["Beginner%","Intermediate%","Advanced%","Expert%"]
T = resorts[["Intermediate%","Advanced%"]].values
resorts["u_terrain"] = 0.25 * (T[:,0] + T[:,1])
resorts = resorts.drop(columns = ['skiable_acres', 'Weekly Pass Price (AUD)', 'Direct Public Coach', 
                                  'Beginner%', 'Intermediate%', 'Advanced%', 'Expert%'])
resorts
                       

,ski_resort_name,u_size,u_price,u_access,u_terrain
0,Falls Creek Ski Resort,0.981600,0.500000,1.0,0.2075
1,Hotham Ski Resort,0.974047,0.384635,1.0,0.2025
2,Mt Buller Ski Resort,0.972422,0.484681,1.0,0.2300
3,Perisher Ski Resort,1.000000,0.469391,1.0,0.1950
4,Thredbo Ski Resort,0.983180,0.484681,1.0,0.2100
5,Mt Baw Baw Ski Resort,0.000000,0.880422,0.0,0.1875
6,Selwyn Ski Resort,0.854706,0.765849,0.0,0.1575
7,Charlotte Pass Ski Resort,0.876235,0.566014,0.0,0.0700
8,Mt Stirling Ski Resort,NaN,0.904430,0.0,0.2000


In [81]:
name_map_resorts = {
    "Falls Creek Ski Resort": "Falls_Creek",
    "Hotham Ski Resort": "Mt._Hotham",
    "Mt Buller Ski Resort": "Mt._Buller",
    "Perisher Ski Resort": "Perisher",
    "Thredbo Ski Resort": "Thredbo",
    "Mt Baw Baw Ski Resort": "Mt._Baw_Baw",
    "Selwyn Ski Resort": "Selwyn",
    "Charlotte Pass Ski Resort": "Charlotte_Pass",
    "Mt Stirling Ski Resort": "Mt._Stirling"
}

# Add a new column in static for merging
resorts["M_name"] = resorts["ski_resort_name"].replace(name_map_resorts)

Hotham_copy = Hotham.merge(resorts, on="M_name")
Hotham_copy['snow_utility'] = snow.loc[snow["M_name"] == "Mt Hotham", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Hotham_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Hotham_copy["topsis"] = topsis_scores(Z, w)
result = Hotham_copy[['ds','topsis']]
result

Entropy weights: {'visitor_utility': 0.020877367181014062, 'snow_utility': 0.025173117025314183, 'u_size': 0.23848737894841793, 'u_price': 0.23848737894841793, 'u_access': 0.23848737894841793, 'u_terrain': 0.23848737894841793}


,ds,topsis
0,2025-06-02,0.619727
1,2025-06-09,0.423079
2,2025-06-16,0.374303
3,2025-06-23,0.449548
4,2025-06-30,0.162676
5,2025-07-07,0.059991
6,2025-07-14,0.276701
7,2025-07-21,0.443975
8,2025-07-28,0.404628
9,2025-08-04,0.359199


In [85]:
Fc_copy = Fc.merge(resorts, on="M_name")

Fc_copy['snow_utility'] = snow.loc[snow["M_name"] == "Falls Creek", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Fc_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Fc_copy["topsis"] = topsis_scores(Z, w)

result['topsis'] = result['topsis'] + Fc_copy['topsis']
result

Entropy weights: {'visitor_utility': 0.025175973089832045, 'snow_utility': 0.028414212905016334, 'u_size': 0.23660245350128786, 'u_price': 0.23660245350128786, 'u_access': 0.23660245350128786, 'u_terrain': 0.23660245350128786}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/3082509492.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + Fc_copy['topsis']


,ds,topsis
0,2025-06-02,1.616509
1,2025-06-09,1.104179
2,2025-06-16,0.902755
3,2025-06-23,0.883651
4,2025-06-30,0.404399
5,2025-07-07,0.167794
6,2025-07-14,0.454266
7,2025-07-21,0.734322
8,2025-07-28,0.715698
9,2025-08-04,0.624595


In [87]:
Charlotte_copy = Charlotte.merge(resorts, on="M_name")

Charlotte_copy['snow_utility'] = snow.loc[snow["M_name"] == "Charlotte Pass", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Charlotte_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Charlotte_copy["topsis"] = topsis_scores(Z, w)
result['topsis'] = result['topsis'] + Charlotte_copy["topsis"]

Entropy weights: {'visitor_utility': 0.021343741980066843, 'snow_utility': 0.02758216983685895, 'u_size': 0.23776852204576854, 'u_price': 0.23776852204576854, 'u_access': 0.23776852204576854, 'u_terrain': 0.23776852204576854}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/630563572.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + Charlotte_copy["topsis"]


In [91]:
BB_copy = BB.merge(resorts, on="M_name")

BB_copy['snow_utility'] = snow.loc[snow["M_name"] == "Mt Baw Baw", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(BB_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
BB_copy["topsis"] = topsis_scores(Z, w)
result['topsis'] = result['topsis'] + BB_copy["topsis"]

Entropy weights: {'visitor_utility': 0.016110864457673225, 'snow_utility': 0.03199049976712372, 'u_size': 0.23797465894380074, 'u_price': 0.23797465894380074, 'u_access': 0.23797465894380074, 'u_terrain': 0.23797465894380074}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/3140651231.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + BB_copy["topsis"]


In [94]:
Thredbo_copy = Thredbo.merge(resorts, on="M_name")

Thredbo_copy['snow_utility'] = snow.loc[snow["M_name"] == "Thredbo", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Thredbo_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Thredbo_copy["topsis"] = topsis_scores(Z, w)
result['topsis'] = result['topsis'] + Thredbo_copy["topsis"]

Entropy weights: {'visitor_utility': 0.007645445828927563, 'snow_utility': 0.0257649773600895, 'u_size': 0.24164739420274572, 'u_price': 0.24164739420274572, 'u_access': 0.24164739420274572, 'u_terrain': 0.24164739420274572}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/2830492680.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + Thredbo_copy["topsis"]


In [98]:
Buller_copy = Buller.merge(resorts, on="M_name")

Buller_copy['snow_utility'] = snow.loc[snow["M_name"] == "Mt Buller", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Buller_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Buller_copy["topsis"] = topsis_scores(Z, w)
result['topsis'] = result['topsis'] + Buller_copy["topsis"]

Entropy weights: {'visitor_utility': 0.01599832902818837, 'snow_utility': 0.022611543106802155, 'u_size': 0.24034753196625236, 'u_price': 0.24034753196625236, 'u_access': 0.24034753196625236, 'u_terrain': 0.24034753196625236}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/741403608.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + Buller_copy["topsis"]


In [103]:
Perisher_copy = Perisher.merge(resorts, on="M_name")

Perisher_copy['snow_utility'] = snow.loc[snow["M_name"] == "Perisher", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Perisher_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Perisher_copy["topsis"] = topsis_scores(Z, w)
result['topsis'] = result['topsis'] + Perisher_copy["topsis"]
result

Entropy weights: {'visitor_utility': 0.019406276506423945, 'snow_utility': 0.01802945892155875, 'u_size': 0.24064106614300432, 'u_price': 0.24064106614300432, 'u_access': 0.24064106614300432, 'u_terrain': 0.24064106614300432}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/1199256240.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + Perisher_copy["topsis"]


,ds,topsis
0,2025-06-02,5.649140
1,2025-06-09,3.080480
2,2025-06-16,2.553252
3,2025-06-23,2.928599
4,2025-06-30,1.557733
5,2025-07-07,1.403951
6,2025-07-14,2.401618
7,2025-07-21,3.253973
8,2025-07-28,3.191375
9,2025-08-04,2.879140


In [113]:
Selwyn_copy = Selwyn.merge(resorts, on="M_name")

Selwyn_copy['snow_utility'] = snow.loc[snow["M_name"] == "Selwyn", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Selwyn_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Selwyn_copy["topsis"] = topsis_scores(Z, w)
result['topsis'] = result['topsis'] + Selwyn_copy["topsis"]

Entropy weights: {'visitor_utility': 0.020153194526749223, 'snow_utility': 0.018015725886722213, 'u_size': 0.2404577698966321, 'u_price': 0.2404577698966321, 'u_access': 0.2404577698966321, 'u_terrain': 0.2404577698966321}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/2838926464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + Selwyn_copy["topsis"]


In [117]:
Stirling_copy = Stirling.merge(resorts, on="M_name")

Stirling_copy['snow_utility'] = snow.loc[snow["M_name"] == "Mt Stirling", "snow_utility"].values

criteria = ["visitor_utility", "snow_utility", "u_size", "u_price", "u_access", "u_terrain"]
Z = np.vstack([minmax01(Stirling_copy[c].values) for c in criteria]).T

# Compute entropy weights
w = entropy_weights(Z)
print("Entropy weights:", dict(zip(criteria, w)))

# Compute TOPSIS closeness scores
Stirling_copy["topsis"] = topsis_scores(Z, w)
result['topsis'] = result['topsis'] + Stirling_copy["topsis"]
result

Entropy weights: {'visitor_utility': 0.01890909261452198, 'snow_utility': 0.022544656171294118, 'u_size': 0.23963656280354595, 'u_price': 0.23963656280354595, 'u_access': 0.23963656280354595, 'u_terrain': 0.23963656280354595}


/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/1329495377.py:2: RuntimeWarning: All-NaN slice encountered
  mn, mx = np.nanmin(x), np.nanmax(x)
/var/folders/s3/jl_ql_591ss27s827xmdb6c40000gn/T/ipykernel_7514/2494584235.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['topsis'] = result['topsis'] + Stirling_copy["topsis"]


,ds,topsis
0,2025-06-02,7.515980
1,2025-06-09,4.168610
2,2025-06-16,3.808131
3,2025-06-23,4.802143
4,2025-06-30,3.012812
5,2025-07-07,2.800957
6,2025-07-14,4.188862
7,2025-07-21,5.193480
8,2025-07-28,4.774176
9,2025-08-04,4.271939
